In [ ]:
import pandas as pd                       # Per treballar amb dades en DataFrame
import seaborn as sns                     # Per visualització estadística
import matplotlib.pyplot as plt           # Per visualitzacions generals
import numpy as np                        # Per càlculs numèrics

In [ ]:
# Llegim el dataset
df = pd.read_csv("alzheimers_disease_data.csv")
df

In [ ]:
#Mostra informació de columnes, tipus de dades i valors nuls
df.info()

In [ ]:
# Eliminem columnes que no són útils per l'anàlisi
alzheimer = df.drop(columns=["DoctorInCharge", "PatientID"])
alzheimer

In [ ]:
#Visualització de la variable objectiu
plt.figure(figsize=(6,4))  
ax = sns.countplot(x='Diagnosis', data=alzheimer)
plt.title('Count of Diagnosis')
plt.xticks([0, 1], ['No Alzheimer\'s', 'Alzheimer\'s'])
plt.show()


In [ ]:
#Separació de variables numèriques i categòriques
var_numeriques = [col for col in alzheimer.columns if alzheimer[col].nunique() > 10] #Considerem que les variables amb més de 10 valors diferents com variables numeriques.
var_categoriques = alzheimer.columns.difference(var_numeriques).difference(['Diagnosis']).to_list()

In [ ]:
# Visualització de variables categòriques respecte el diagnostic
for var in var_categoriques:
    plt.figure(figsize=(6,4))
    sns.countplot(x="Diagnosis", hue=var, data=alzheimer)
    plt.title(f"Count Plot of {var}")
    plt.show()

In [ ]:
# Visualització de variables numèriques respecte la diagnosi
for var in var_numeriques:
    plt.figure(figsize=(8,5))
    sns.boxplot(x='Diagnosis', y=var, data=df)
    plt.title(f'{var} by Diagnosis')
    plt.xticks([0, 1], ['No Alzheimer\'s', 'Alzheimer\'s'])
    plt.show()


In [ ]:
sns.pairplot(df[var_numeriques + ['Diagnosis']], hue='Diagnosis')
plt.show()

In [ ]:
# Heatmap de correlació entre variables numèriques
plt.figure(figsize=(25,25))
sns.heatmap(alzheimer.corr(), annot=True, linewidths=.5, fmt='.3f')
plt.show()

In [ ]:
# Detecció de valors extrems (outliers) amb IQR
for var in var_numeriques:
    Q1 = alzheimer[var].quantile(0.25)
    Q3 = alzheimer[var].quantile(0.75)
    IQR = Q3 - Q1
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR
    outliers = alzheimer[(alzheimer[var] < lower_bound) | (alzheimer[var] > upper_bound)]
    outliers.info() 